# 0.0. Imports

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb

from matplotlib import pyplot as plt
from IPython.display import HTML

from sklearn import model_selection as ms
from sklearn import neighbors as ng
from sklearn import linear_model as lm
from sklearn import ensemble as en

## 0.1. Helper Functions

In [2]:
def precision_at_k(data, k):
    data = data.reset_index(drop=True)
    data['ranking'] = data.index + 1
    
    data['precision_at_k'] = data['response'].cumsum() / data['ranking']
    
    return (data.loc[k, 'precision_at_k'], data)


def recall_at_k( data, k ):
    data = data.reset_index( drop=True )
    data['ranking'] = data.index + 1
    
    data['recall_at_k'] = data['response'].cumsum() / data['response'].sum()
    
    return ( data.loc[ k, 'recall_at_k'], data )


def perfomance_metrics_at_k(x_val, y_val, yhat, model_name, percentage_of_base):  
    df = x_val.copy()
    df['response'] = y_val
    df['score'] = yhat[:, 1].tolist()
    df = df.sort_values( 'score', ascending=False )

    k = int(percentage_of_base * df.shape[0])

    precision,  data = precision_at_k(df, k)
    recall, data = recall_at_k(df, k)

    return pd.DataFrame({'model_name': [model_name],
                         'Precision@K': [round(precision, 3)],
                         'Recall@K': [round(recall, 3)]}, index=[0])
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

In [4]:
df_raw = pd.read_csv('../data/raw/train.csv')

# 1.0. Data Description

In [5]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [6]:
cols_new = ['id', 'gender', 'age', 'driving_license', 'region_code', 'previously_insured', 'vehicle_age', 
            'vehicle_damage', 'annual_premium', 'policy_sales_channel', 'vintage', 'response']
df1.columns = cols_new

# 2.0. Feature Engineering

In [7]:
df2 = df1.copy()

## 2.1. Feature Creation

In [8]:
# gender
gender_map = {'Male': 1, 'Female': 0}
df2.loc[:, 'gender'] = df2['gender'].map(gender_map)

# vehicle_age
vehicle_age_map = {'1-2 Year': 'between_1_2_years', '< 1 Year': 'bellow_1_year', '> 2 Years': 'over_2_year'}
df2.loc[:, 'vehicle_age'] = df2['vehicle_age'].map(vehicle_age_map)

# vehicle_damage
vehicle_damage_map = {'Yes': 1, 'No': 0}
df2.loc[:, 'vehicle_damage'] = df2['vehicle_damage'].map(vehicle_damage_map)

# 3.0. Data Filtering

In [9]:
df3 = df2.copy()

# 4.0. EDA

In [10]:
df4 = df3.copy()

# 5.0. Data Preparation

In [11]:
X = df4.drop(['response', 'id'], axis=1).copy()
y = df4['response'].copy()

x_train, x_validation, y_train, y_validation = ms.train_test_split(X, y, test_size=0.2)

df5 = pd.concat([x_train, y_train], axis=1)

## 5.1. Standardization

## 5.2. Rescaling

## 5.3. Encoding

In [12]:
df5 = pd.get_dummies(df5, columns=['vehicle_age'], prefix=['vehicle_age'])

## 5.4. Data Preparation - Validation 

In [13]:
x_validation = pd.get_dummies(x_validation, columns=['vehicle_age'], prefix=['vehicle_age'])

# 6.0. Feature Selection

In [14]:
cols_selected = ['gender', 'age', 'driving_license', 'region_code', 
                 'previously_insured','vehicle_damage', 'annual_premium', 
                 'policy_sales_channel', 'vintage', 'vehicle_age_bellow_1_year',
                 'vehicle_age_between_1_2_years', 'vehicle_age_over_2_year']

# 7.0. Machine Learning Modelling

In [15]:
x_train = df5[cols_selected]

x_val = x_validation[cols_selected]
y_val = y_validation

## 7.1. KNN

In [16]:
# model definition and fit
model_knn = ng.KNeighborsClassifier().fit(x_train, y_train)

# Predictions
yhat_knn = model_knn.predict_proba(x_val)

# Performance
result_knn = perfomance_metrics_at_k(x_val, y_val, yhat_knn, 'KNeighborsClassifier', 0.2)
result_knn

## 7.2. Logistic Regression

In [28]:
# model definition and fit
model_lr = lm.LogisticRegression().fit(x_train, y_train)

# predictions
yhat_lr = model_lr.predict_proba(x_val)

# Performance
result_lr = perfomance_metrics_at_k(x_val, y_val, yhat_lr, 'LogisticRegression', 0.2)
result_lr

/home/cid/.pyenv/versions/3.8.0/envs/health-insurance-cross-sell-prediction/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,model_name,Precision@K,Recall@K
0,LogisticRegression,0.228,0.375


## 7.3. Extra Trees Classifier

In [23]:
# model definition and fit
model_et = en.ExtraTreesClassifier().fit(x_train, y_train)

# predictions
yhat_et = model_et.predict_proba(x_val)

# Perfomance
result_et = perfomance_metrics_at_k(x_val, y_val, yhat_et, 'ExtraTreesClassifier', 0.2 )
result_et

## 7.4. Random Forest Classifier

In [26]:
model_rf = en.RandomForestClassifier().fit(x_train, y_train)

yhat_rf = model_rf.predict_proba(x_val)

result_rf = perfomance_metrics_at_k(x_val, y_val, yhat_rf, 'RandomForestClassifier', 0.2)
result_rf

## 7.5.  XGBoost Classifier

In [35]:
# model definition and fit
model_xgb = xgb.XGBClassifier().fit(x_train, y_train)

# predictions
yhat_xgb = model_xgb.predict_proba(x_val)

# Perfomance
result_xgb = perfomance_metrics_at_k(x_val, y_val, yhat_xgb, 'XGBClassifier', 0.2)
result_xgb

,model_name,Precision@K,Recall@K
0,XGBClassifier,0.354,0.581


## 7.6. Machine Learning Model's Performance

In [39]:
result = pd.concat([result_knn, result_lr, result_et, result_rf, result_xgb]).sort_values('Recall@K', ascending=False)
result

,model_name,Precision@K,Recall@K
0,XGBClassifier,0.354,0.581
0,RandomForestClassifier,0.320,0.526
0,ExtraTreesClassifier,0.316,0.519
0,LogisticRegression,0.228,0.375
0,KNeighborsClassifier,0.183,0.301
